# S11 T01: Practicant amb training i test sets

## Exercici 1
Parteix el conjunt de dadesDelayedFlights.csv en train i test. Estudia els dos conjunts per separat, a nivell descriptiu.

##Exercici 2
Aplica algun procés de transformació (estandarditzar les dades numèriques, crear columnes dummies, polinomis...).

## Exercici 3
Resumeix les noves columnes generades de manera estadística i gràfica